In [1]:
# Import 設定
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parents[0]
sys.path.insert(0, PROJECT_DIR.joinpath("src").__str__())  # 不能用 pathlib.PosixPath 傳入

# Step 01: Ingestion

In [2]:
from ingestion.file_loaders.goodnotes.loader import GoodnotesLoader
loader = GoodnotesLoader()

loader_results = []
for filepath in ["早安日語-孫寅華.pdf",]:
    print(filepath)
    loader_results = loader.load(filepath)

/home/kevin/.local/share/virtualenvs/personal-rag-s54CZC9a/lib/python3.11/site-packages/paddle/utils/cpp_extension/extension_utils.py:717: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/kevin/.paddlex/official_models/PP-OCRv5_server_det`.
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/kevin/.paddlex/official_models/PP-OCRv5_server_rec`.


早安日語-孫寅華.pdf


/home/kevin/.local/share/virtualenvs/personal-rag-s54CZC9a/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (96677724 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/kevin/.local/share/virtualenvs/personal-rag-s54CZC9a/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (96677724 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/kevin/.local/share/virtualenvs/personal-rag-s54CZC9a/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (96677724 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/kevin/.local/share/virtualenvs/personal-rag-s54CZC9a/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (93500000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  war

In [3]:
loader_results[:100]

[LoaderResult(content='早安日語\n孫寅華', metadata=GoodnotesMetadata(file_type=<FileType.PDF: 'pdf'>, file_name='早安日語-孫寅華.pdf', title=None, author=None, subject=None, created_at=None, modified_at=None, is_chunk=False, chunk_info=None, extra={'bg_mode': 'black'}, source=None, producer='pypdf', page=1, outlines=[]), doc=None),
 LoaderResult(content='早安日語\n孫寅華\n\n二\n▲\n★\n●\n0\n?\n\n\n-\n1', metadata=GoodnotesMetadata(file_type=<FileType.PDF: 'pdf'>, file_name='早安日語-孫寅華.pdf', title=None, author=None, subject=None, created_at=None, modified_at=None, is_chunk=False, chunk_info=None, extra={'bg_mode': 'black'}, source=None, producer='pypdf', page=1, outlines=[]), doc=None),
 LoaderResult(content='本文\n', metadata=GoodnotesMetadata(file_type=<FileType.PDF: 'pdf'>, file_name='早安日語-孫寅華.pdf', title=None, author=None, subject=None, created_at=None, modified_at=None, is_chunk=False, chunk_info=None, extra={'bg_mode': 'black'}, source=None, producer='pypdf', page=2, outlines=['Note']), doc=None),
 LoaderRe

# Step 2: Chunk

In [4]:
from chunking.no_chunk import NoChunkProcessor
chunker = NoChunkProcessor()

chunks = []
for loader_result in loader_results:
    if not loader_result.content:
        continue
    chunks += chunker.process(doc=loader_result.content,
                              metadata=loader_result.metadata,
                              )

### 手動將 Outline 加入 Chunk

In [ ]:
for idx in range(len(chunks)):
    chunks[idx].metadata.file_name += (f" Page{chunks[idx].metadata.page:03d}")

### 手動調整頁碼

In [ ]:
# import re
# for idx in range(len(chunks)):
#     filename = chunks[idx].metadata.file_name
#     if filename.startswith("早安日語-孫寅華"):
#         regex = re.search(r"早安日語-孫寅華part(\d+)-page(\d+)", filename)
#         if regex:
#             part_num = int(regex.group(1))
#             page_num = int(regex.group(2))
#     chunks[idx].metadata.page += (part_num-1)*page_num


In [22]:
len(chunks)

472

In [23]:
print(chunks[123])

id=UUID('0510b9fc-059d-5f92-981a-6a1916691ab1') content='*授受動詞：AはBに～をV\n(lesson 80~82)\n*(Aは)Bに~をあげる\nさしあげる\n句子+つもり(表目的)\n注意句尾照N的连接\n*子使用时机(以下可以用代替“あげ)\n在對外人提起“给家人~時\n給動物，植物，某物時\n(A给B~)\n表原因\nガ表原因\n完整句子\n7形+ガ\n表(動作)先後\n時/地+から\n表時間/地点起桌\n*Recall:分\n古くなる→古くなた\n*注意使用时机\n表原因\n注意使用时机\n*(Aは)Bに/ガ~をも\n(A從B那收到~)\nいただく\n*のて(表原因)\n*よくでまた(做得好)\n*力5 v.S.07\n5：主\n表原因(较客觀)\n考得很好\n*Aが~をくれる\n(A给我~)\n重体修饰\n形式名詞，代替7タ\n*以下介绍待遇表(…\n1.あけ(我/人做什麼)\nNore：通常只用於熟人，家人；對陌生人有另外的詞组以教)\n带有“我了你…的含意，所以少用\nAternaive用~的邀的来握婉表\n2. (CH19)用謙讓語表達\n2.…(人/什)\n(…て<ださる)\n(我買書給了林士人/我為林士人買了書)\n(林士人為我做了蛋糕)\n3.…もらう(讓誰/幫了我們做了什麼\n*可將「見てもう」當慣用語\n(因為生病，我去看醫生)\n(我讓醫生慧我看)\nrecall:醫生的敬稱\n但面對面稱呼醫生時用先生，\n可視為慣用語\n表客觀原因\n*授受動詞進階\n*Rcall:あげ v.s.<\nあげ子：我方给對方/他人給他人\n<机3：他方给我方\n在對話情境中\n*如果接受人是我方的人，也可以用\nEx：先生は()辞書妹に<だいまし。(老師妹一本好字典\n我妹、己方的.\nにお土\n*「あげる」くれる對家之係\n用げ两墨都是外人」关子的妹妹\nEx:关子关)\n用妹是自己，自己的\nNote:由於不是向咲子說話\n所以不用稱“关子的妹妹，妹士人' metadata=GoodnotesMetadata(file_type=<FileType.PDF: 'pdf'>, file_name='早安日語-孫寅華.pdf Page 065', title=None, author=None, s

# Step 3: Embedding

In [24]:
import os
from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel


embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
for idx in range(len(chunks)):
    vector = embedder.encode(chunks[idx].content)
    chunks[idx].embedding = vector

In [25]:
chunks[idx]

Chunk(id=UUID('739a42b9-da3f-5c20-b213-e8507d7c24db'), content='ぼうず\nてるてる坊主\n浅原鏡村\n中山晋平\n作詞\n作曲\nd= 69\nてるてるぼう\n9\nず\nてるぼ\nう\nず\nあ\nしたてん\nきに\n©\n0\nしおくれ\nい\nつ\nか\nの\np\nめ\nの\n-\nの\n。\nd\nに\nす\nそ\n一\nは\n一\nら\nよう\nれーた\nら\n-\n一\nげ\n一\n上\nき\n一\nん\nの\nす\nず\n一\nあ\nぼうず\nぼうず\nてるてる坊主\nてる坊主\nゆめ\nしておくれいつかの夢の\nは\nそら\nきん\nすず\n空のように晴れたら金の鈴あげよ\nてんき\nあした天気に\n晴天娃娃\u3000晴天娃娃\n讓明天天氣變好吧\n像哪天夢中的天空一樣\n果真轉晴的話\n就送你金色鈴鐺喲\n晴天娃娃晴天娃娃\n讓明天天氣變好吧\n如果你聽到我許的願\n我就給你喝很多甜美的酒哦\n晴天娃娃晴天娃娃\n讓明天天氣變好吧\n如果還是烏雲滿佈\n天空又繼續哭泣的話\n就剪斷你的頭哦\nてんき\nぼうず\nぼうず\nてるてる坊主\u3000てる坊主\u3000あした天気に\nねが\nき\nわたし\nしておくれ私の願いを聞いたなら\nさけ\nの\nあまいお酒を\u3000たんと飲ましょ\nぼうず\nてんき\nぼうず\nてるてる坊主\u3000てる坊主\u3000あした天気に\nくも\nしておくれそれでも曇って\nキ\nくび\nな\n泣いてたらそなたの首をチョンと切るぞ\nCD1-07\n17', metadata=GoodnotesMetadata(file_type=<FileType.PDF: 'pdf'>, file_name='早安日語-孫寅華.pdf Page 251', title=None, author=None, subject=None, created_at=None, modified_at=None, is_chunk=True, chunk_info={'chunk_index': 0, 'total_chunks': 1, 'chunk_size': 505}, extra={'bg_mode': 'white'}, source=None, pr

# Step 04: Insert

## Vector-based

In [33]:
from infra.stores.pgvector import PGVectorStore

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )

for chunk in chunks:
    vec_store.insert(chunk)

## Lexical-based

In [34]:
from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )

for chunk in chunks:
    lex_store.insert(chunk)

# Step 05: Retrieve

## Vector-based

In [35]:
import os

from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel
from infra.stores.pgvector import PGVectorStore

embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
query_vector = embedder.encode("つつある")

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )
results = vec_store.search(query_vector)

In [36]:
[_.chunk.content for _ in results]

['*気配/配为(關心，專心，注意，他人考虑\n~に気配りをする/~に気を配る)在~方面為他人著想\n=配慮や注意どを行主渡らせる\n(他五>分配(關心)\nEx:资料飞配子(發料)\n配)\n(lesson155156)\n人が大勢出たりてやがになること\n二客多<入7，商上行\n顺利;成功\n<自五>熟關;擁擠(;生意興隆)\n夫外出い状態にる\n十分入てい13\n<自五)充满；(感情)深厚(;被装在其中，無法跑出去)\nも行ガない\n家家中，足不出户)\n心がこもる\n心のこも\n手紙(滿懷感情的信\n*賑わう：(熱鬧)りで町も賑わっています\n(生意好)店が賑わう\n(他五>編織\nEx:セ-ターを編みます\n<他五>讓給;傳給(;轉賣;留待日後)\n:座を讓る(讓座)\n可以用来隱酶的表示赏\n安(讓る(把車便宜轉索)\n(待日後)\n日譲る\n(形動)獨特;唯\n(unique)\n(~にモテる:受到~歡迎)\nv.s.持てる/モ<白下一>有人缘.受歡迎\nもてはやす\n<名·形動>很有人緣；深受歡迎\nEx：手紙のやリ取り(書信往来)\n<名·他升>互赠;往来\nEx-日の人びリとす\n<副·自サ>悠閣地;無拘無束地\n×通体三日制が定着した。\n<名·自>固定下来\n元気なく寂しそうに\n<副·自升)重頭丧氣\nEx：会議は円滑に運んだ\n搬遇，亦可做\n(名.形動>圖滑；順利\n进行\n二物事がとどこおらすに進む\n(名)情人節(St Vabtieda)\n<名>白色情人 (white day))\n<慢)格好(加)：形容男性英俊，\n现在不太用八」\n<名>外表；形状;体面\n(名>義理巧克力\n<名)本命巧克力\n<名>包装材料（wrapping)\n<名>店面;門市\n<名>外表;表面上看来\n一般認知上的圍巾\n<名>圍巾 (muffler\n機車/汽車消音器\nEx:子供度孩子不一定好\nいか悪いかが簡に決められたい\n(名)是非，好恶(;無法轻易判断好壤的事物)\n(名)情書 (love eter)\n襟卷\n(名)事件;節日 (event)\n<名>答謝的禮物\n<名)壶;罐;谭子\n<名>服飾配件；節品 (accessories)\n同时参考上方配\n為男性倒茶\n=気を能子こ\n(名)泡茶;斟茶\n(名)多方注意;照顧\n*気配/

## Lexical-based

In [37]:
import os

from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )
results = lex_store.search("つつある")

In [38]:
[_.chunk.content for _ in results]

['よみもの\nはい\nにほんじん\nみ\nこたつに入ってみかんを食べながら、テレビを見るこれは日本人の\nす\n不ゆ\nかた\nだいひようてき\n（\nき\nふとん\nうえ\n代表的な冬の過ごし方す。こたつは木を組んぐらの上に布団ぶ\n*注意此處分ぶせる.s.のせる\nつくえいた\nなかだんほうきぐ\nほ\nい\nガぶせる：從上往下罩住\n→のせる：放在上面\nせて、机の板をのせて、中に暖房器具を入れたものす。こたつには掘り\nお\nにしゆるい\nゆかひくほ\nほ\nごたつと置きごたつの二種類があります。掘りごたつは、床を低く掘って\nあし\nなかたんぼうき\nの\nはい\n中に暖房器具を入れます。足を伸ばしたま、こたつに入ることができま\n記-下\nつか\nなつあいだ\nいま\nいた\nす。夏の間こたつを使わないときには、板をかぶせます。しかし、今では\nつかひとほお\nてかる。\nお\nお\nより手軽な置きごたつを使う人の方が多いです。置きごたつには、やぐら\n(副)：更~(程度更甚)\nだんぼうきぐ\nうら\nの裏に暖房器具がついています。\nへやぜんたいあたた\nからだ、あたた\nこたつは体を暖めることができますが、部屋全体を暖めることができま\n使~暖和\nげんさい\nひじようさむちほう\nつか\nほっかいどう\nせん。ですから、北海道など非常に寒い地方ではあまり使いません。現在\nだんぼうきしゆいおお\nでんき\nはヒーター、ストーブ電気カーペットなど暖房器具の種類が多くなりま\nあたた\nばしよ\nあい\nかぞくあつ\nした。しかし、こたつは家族が集まってくつろいで暖まる場所として、相\n<自五>表\nか\nにんき\n変わらず人気があります。\n有人気\n<副)仍然\n置きごたつ\n掘りごたつ\n机の板\n·布団·\n暖房器具\nCD1-21\n37',
 'インターチェンジ\nはい\nあめ\nふ\nつづ\nむあつ\nかん\nつゆに入って雨が降り続くと、蒸し暑くてしい感じになる。\nとき\nぼうず\nつく\nつき\nねが\nひ\nせいてん\nそんな時は、「てる坊主」を作っ、次の日の晴天をお願いしてみ\nぼうずかみぬのつく\nあし\nにんぎょう\nよう。てるてる坊主は紙か布で作れた、手も足もない人形こと。\nこども\nせいてん\nねが\nあしたてんき\